# Actor-critic learning for the CVRP

## Importing the libraries

In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Creating a CVRP environment

In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
import random
import copy
import math

class VRPEnv(gym.Env):
  def __init__(self):
    # customer count ('0' is depot) 
    self.customer_count = 11
    # the capacity of vehicles
    self.vehicle_capacity = 2
  
    self.action_space = spaces.Discrete(3)
    self.observation_space = spaces.Box(low=0,high=1, shape=(4,1), dtype=np.float64)
    self.VRP = np.array((self.customer_count,4))
    self._max_episode_steps = 1000
    self.viewer = None
    self.state = None
    self.steps_beyond_done = None
    self.route = []
    self.route.append(0)
    self.previous_action = 0
    self.hn_actor = torch.zeros([1,self.customer_count,128], dtype=torch.float32)
    

  def reset(self, seed=200):
    if seed == 200:
      seed = int(time.time())
    np.random.seed(seed)
    x_locations = (np.random.rand(self.customer_count)).reshape((self.customer_count,1))
    y_locations = (np.random.rand(self.customer_count)).reshape((self.customer_count,1))
    demand = (np.random.randint(1,9,self.customer_count).reshape((self.customer_count,1))).reshape((self.customer_count,1))/10 # Normalise to between 0.1 and 0.9
    capacity = np.repeat(self.vehicle_capacity,self.customer_count).reshape((self.customer_count,1))
    VRP = np.concatenate((np.concatenate((np.concatenate((x_locations,y_locations), axis=1),demand),axis=1),capacity),axis=1)
    self.VRP = VRP.reshape((self.customer_count,4))
    self.unserved_customers = []
    for i in range(1, self.customer_count):
      self.unserved_customers.append(i)
    self.routes = []
    self.route = []
    self.route.append(0)
    self.VRP[0,2] = 0 # Set the demand at thedepot to 0
    self.state = copy.deepcopy(self.VRP)
    self.previous_action = 0
    return self.state
  

  def step(self, action):
    # Calculate the reward as the negative euclidean distance
    reward = -((self.state[self.previous_action,0]-self.state[action,0])**2+(self.state[self.previous_action,1]-self.state[action,1])**2)**0.5 # - Euclidean distance between customers
    load = self.state[0,3]
    self.state[:,3] = max(0,(load-self.state[action,2])) # Update the vehicle load
    self.state[action, 2] = max(0,self.state[action,2]-load) # Update the demand at served customer
    done = False
    if action == 0: # Return to the depot
      self.route.append(action) # End route
      self.routes.append(self.route) # Add subroute to list of all routes
      self.route = [] # Initiate new subroute
      self.state[:,3] = self.vehicle_capacity # Refill the vehicle
    self.route.append(action) # Add action to the subroute
    if max(self.state[:,2]) > 0: # If there are unserved customers left
      done = False
    elif max(self.state[:,2]) == 0 and action == 0: # If there are no unserved customers left and we have returned to the depot
      done = True
      self.route.append(0)
    self.previous_action = action # Update the previous action
    return self.state, reward, done


## Let's test the environment step function

In [3]:
env = VRPEnv() # Create an instance of the environment
state = env.reset() # Reset the environment
action = 2 # Perform action with customer 2
print(state)
state, reward, done = env.step(action) # Perform the actual transition
print(state)

[[0.69039862 0.06613502 0.         2.        ]
 [0.62200451 0.78504387 0.7        2.        ]
 [0.26096926 0.09412515 0.7        2.        ]
 [0.43824386 0.48512059 0.6        2.        ]
 [0.50067945 0.88418096 0.1        2.        ]
 [0.92344961 0.51373972 0.2        2.        ]
 [0.18505191 0.87495518 0.5        2.        ]
 [0.09287534 0.152164   0.2        2.        ]
 [0.35425302 0.32937999 0.7        2.        ]
 [0.7271695  0.295905   0.7        2.        ]
 [0.62360697 0.36830169 0.6        2.        ]]
[[0.69039862 0.06613502 0.         1.3       ]
 [0.62200451 0.78504387 0.7        1.3       ]
 [0.26096926 0.09412515 0.         1.3       ]
 [0.43824386 0.48512059 0.6        1.3       ]
 [0.50067945 0.88418096 0.1        1.3       ]
 [0.92344961 0.51373972 0.2        1.3       ]
 [0.18505191 0.87495518 0.5        1.3       ]
 [0.09287534 0.152164   0.2        1.3       ]
 [0.35425302 0.32937999 0.7        1.3       ]
 [0.7271695  0.295905   0.7        1.3       ]
 [0.62360697

## Initialize the Experience Replay memory

In [4]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      #self.storage[int(self.ptr)] = transition
      #self.ptr = (self.ptr + 1) % self.max_size
      self.storage.pop(0)
      self.storage.append(transition)
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind =  np.arange((len(self.storage)-(batch_size+1)),len(self.storage)-1,1) #np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Build the neural network for the Actor and Actor-target models - contains the attention mechanism

In [5]:
class Actor(nn.Module):
  
  def __init__(self, state_dim=4, embed_size = 128):#, action_dim, max_action):
    super(Actor, self).__init__()
    self.embed = nn.Linear((state_dim), embed_size) # Encoding to higher dimensional space - can also be changed to convolutional layer as in the paper
    self.u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the attention mechanism
    self.v_t_a = nn.Linear(embed_size,1) # Linear for getting u_t
    self.bar_u_t = nn.RNN(embed_size,embed_size,1) # RNN Layer for the context vector
    self.a_t = nn.Softmax(dim = 1) # Softmax layer for the attention mechanism
    self.v_t_u = nn.Linear(embed_size,1) # Linear for getting u_t
    self.final = nn.Softmax(dim = 1) # Softmax layer for the final output

  def forward(self, x, hn = env.hn_actor):
    cond1 = (x[:,:,2]<x[:,:,3]).int() # Can we meet the demand
    cond2 = (x[:,:,2]>0).int() # Is there demand at the customer
    mask1 = torch.minimum(cond1,cond2) # Select those customers with demand, and whose demand we can meet
    mask1 = torch.reshape(mask1,(len(x),env.customer_count,1))
    x = self.embed(x)
    u, hn = self.u_t(x, hn)
    u = self.v_t_a(u)
    a = self.a_t(u) # Up to equation (4) now
    c = torch.randn(x.shape)
    c = torch.mul(x,a)
    c = torch.sum(c, 0)
    c = torch.reshape(c,(1,env.customer_count,128))
    u_bar, hu = self.bar_u_t(x,c)
    u_bar = self.v_t_u(u_bar)
    output = self.final(u_bar)
    output = torch.mul(output,mask1)
    return output

## Build the neural network for the Critic and Critic-target model

In [6]:
class Critic(nn.Module):
  
  def __init__(self, state_dim=4, action_dim = env.customer_count, embed_size = 128):
    super(Critic, self).__init__()
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim, embed_size) # Perform the embedding
    self.layer_2 = nn.Linear(embed_size, embed_size) # Adding the single dense layer
    self.layer_3 = nn.Linear(embed_size, 1) # Adding the output layer


  def forward(self, x, u): # x is the state, u is the action
    # Forward-Propagation on the Critic Neural Network
    x1 = F.relu(self.layer_1(x))
    ws = torch.mul(x1,u)
    x2 = F.relu(self.layer_2(ws))
    x2 = self.layer_3(x2)
    x2 = torch.sum(x2,1)
    return x2


## Testing the actor and critic networks to confirm their output

In [7]:
env = VRPEnv()
env.reset()
state = env.reset()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state = torch.Tensor(state.reshape(1,env.customer_count,4)).to(device)
actor = Actor()
actor_target = Actor()
prediction = actor(state)
prediction_target = actor_target(state)
print(prediction)
critic = Critic()
q_value = critic(state,prediction)
print(q_value)

AttributeError: module 'torch' has no attribute 'minimum'

## Training Process

In [ ]:
# Selecting the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.autograd.set_detect_anomaly(True)

# Building the Training Process into a class
class Actor_Critic(object):
  
  def __init__(self, state_dim):
    self.actor = Actor(state_dim).to(device)
    self.actor_target = Actor(state_dim).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr = 0.0001)
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state, state_dim):
    state_tensor = torch.Tensor(state.reshape(1,env.customer_count,state_dim)).to(device)
    current_Q = actor(state_tensor).detach()
    current_Q = current_Q.detach().numpy().reshape(env.customer_count)
    action = np.argmax(current_Q)
    return action, current_Q

  def select_target_action(self, state):
    state_tensor = torch.Tensor(state.reshape(1,env.customer_count,state_dim)).to(device)
    target_Q = actor_target(state_tensor).detach()
    target_Q = target_Q.detach().numpy().reshape(env.customer_count)
    action = torch.argmax(target_Q)
    return action, target_Q

  def train(self, replay_buffer, iterations, batch_size=32, discount=0.99, tau=0.005):
    
    for it in range(iterations):
      
      # Sample a batch of transitions (s, s’, a, r) from the memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      # Determine the next action based on the actor target
      next_action = self.actor_target(next_state)
      
      # Determine the target Q-value based on the critic target
      target_Q = self.critic_target(next_state, next_action)

      # Get the final target using the RL update rule
      target_Q = reward + ((1 - done) * discount * target_Q).detach()

      # Determine the probabilities of the actions chosen by the actor
      action = self.actor(state)

      # Determine the current Q-value based on the critic
      current_Q = self.critic(state,action)

      # Compute the loss coming from the critic models: Critic Loss = MSE_Loss(Q, Qt)
      critic_loss = F.mse_loss(current_Q, target_Q)

      # Backpropagate this Critic loss and update the parameters of the Critic model with Adam optimizer
      self.critic_optimizer.zero_grad()
      critic_loss.backward(retain_graph=True)
      self.critic_optimizer.step()

      # Update our Actor model by performing gradient descent on the output of the Critic model  
      actor_loss = -self.critic(state, action).mean()
      self.actor_optimizer.zero_grad()
      actor_loss.backward()
      self.actor_optimizer.step()

      # Update the weights of the Actor target by polyak averaging
      for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

      # Update the weights of the Critic target by polyak averaging
      for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  # Make a save method to save a trained model
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Making a load method to load a pre-trained model
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))


## Create a function that evaluates the policy by calculating its average reward over 10 episodes

In [ ]:
def evaluate_policy(policy, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action, current_Q = policy.select_action(obs, state_dim)
      obs, reward, done = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

## Set the parameters

In [ ]:
env_name = "CVRP" # Name of a environment (set it to any Continous environment you want)
seed = 0 # Random seed number
start_timesteps = 1e4 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
batch_size = 128 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.001 # Target network update rate

## Create a file name for the two saved models: the Actor and Critic models

In [ ]:
file_name = "%s_%s_%s" % ("Actor_Critic", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

## Create a folder to save the trained models

In [ ]:
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

## Create an instance of the CVRP environment

In [ ]:
env = VRPEnv()

## Set seeds and get the necessary information on the states and actions in the chosen environment

In [ ]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.customer_count
max_action = 1

## Create the policy network

In [ ]:
print(state_dim, action_dim, max_action)
policy = Actor_Critic(state_dim)

## Create the Experience Replay memory

In [ ]:
replay_buffer = ReplayBuffer()

## Define a list where all the evaluation results over 10 episodes are stored

In [ ]:
evaluations = [evaluate_policy(policy, eval_episodes=1)]
print(env.routes)

In [ ]:
print(env.routes)
print(env.state)

## Create a folder directory in which the final results will be saved

In [ ]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps
save_env_vid = False
if save_env_vid:
  env = wrappers.Monitor(env, monitor_dir, force = True)
  env.reset()

## Initialize the training process variables

In [ ]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

## Training

In [ ]:
# We start the main loop over 500,000 timesteps
env.reset()
total_timesteps = 0
obs = copy.deepcopy(env.reset())
maximum = -1000
while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:
    
    # If we are not at the very beginning, we start the training process of the model
    if total_timesteps != 0 and total_timesteps > batch_size:
      print("Total Timesteps: {} Episode Num: {} Reward: {} Epsilon: {}".format(total_timesteps, episode_num, episode_reward, epsilon))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      print("Total timesteps: {} Epsilon: {}".format(total_timesteps, min(1,epsilon)))
      evaluations.append(evaluate_policy(policy))
      if evaluations[len(evaluations)-1] > maximum:
        policy.save(file_name, directory="./pytorch_models")
        maximum = evaluations[len(evaluations)-1]
      np.save("./results/%s" % (file_name), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = copy.deepcopy(env.reset())
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Work with epsilon-greedy
  epsilon = 50000/(total_timesteps+1)
  np.random.seed(total_timesteps)
  if np.random.rand() < min(1,epsilon):
    feasible = np.array([0])
    for i in range(1,env.customer_count):
      if obs[i,2] < obs[i,3] and obs[i,2] != 0:
        feasible = np.concatenate((feasible,np.array([i])))
    if len(feasible) > 1:
      feasible = np.delete(feasible,0)
    action = np.random.choice(feasible)
  else: # Choose greedy action
    action, current_Q = policy.select_action(obs, state_dim)
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done = env.step(action)

  # Check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)

  # Increase the total reward
  episode_reward += reward

  # Store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, copy.deepcopy(new_obs), action, reward, done_bool)) # Have to use deepcopy to prevent overwriting of historic next states

  # Update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = copy.deepcopy(new_obs)
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# Add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

## The inference policy function

In [ ]:
def evaluate_final_policy(policy,random = 1, eval_episodes=1):
  avg_reward = 0.
  distance = 0.
  for k in range(eval_episodes):
    obs = env.reset(seed = random)
    done = False
    while not done:
      action, current_Q = policy.select_action(obs, state_dim)
      obs, reward, done = env.step(action)
      avg_reward += reward
    total_distance = 0
    for j in range(len(env.routes)):
      for i in range(len(env.routes[j])-1):
        total_distance = total_distance + math.floor(((env.VRP[env.routes[j][i],0]-env.VRP[env.routes[j][i+1],0])**2+(env.VRP[env.routes[j][i],1]-env.VRP[env.routes[j][i+1],1])**2)**0.5)
    distance += total_distance
  distance /= eval_episodes
  print ("---------------------------------------")
  print ("Average Distance over the Evaluation Step: %f" % (distance))
  print ("---------------------------------------")
  return distance

## Let's see if we can test the implementation

In [ ]:
results = np.zeros(1)
policy.load(file_name, './pytorch_models/')
for i in range(1):
  evaluations = [evaluate_final_policy(policy, random = i)]
  results[i] = evaluations[0]
print(results)